In [1]:
import os
os.chdir("../src/")
import pandas as pd
import s3fs
import import_inpi as inpi

In [3]:
pm, rep = inpi.import_inpi_s3()

/home/coder/work/data_inpi/src/import_inpi.py:109: DtypeWarning: Columns (7,8,10,11,16,30) have mixed types. Specify dtype option on import or set low_memory=False.
  personnes_morales = pd.read_csv(file_in, sep=";")
/home/coder/work/data_inpi/src/import_inpi.py:111: DtypeWarning: Columns (7,15,17,25,26,32,33,34,35,36,37,38,39,40,41,42,43,45,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  representants = pd.read_csv(file_in, sep=";")


In [4]:
pm.shape

(5001845, 31)

In [5]:
rep.shape

(11142886, 51)

In [41]:
import numpy as np
def clean_rep_inpi(df_rep):
    """_summary_Function that cleans data from Inpi

    Args:
        df_rep (_type_): _description_

    Returns:
        _type_: _description_
    """
    # nettoyage du nom des colonnes
    df_rep.columns = [c.lower() for c in df_rep.columns]
    df_rep = df_rep.rename({"code greffe": "code_greffe", 
                                "forme_juridique_x": "forme_juridique", 
                                "prénoms": "prenoms", "qualité": "qualite", 
                                "dénomination": "denomination",
                                "nationalité": "nationalite",
                                "id_représentant": "id_representant",
                                "siren.1": "siren_representant_pm"}, axis=1)
    # suppression de colonnes inutiles
    df_rep = df_rep.drop(['rep_perm_nom',
       'rep_perm_nom_usage', 'rep_perm_pseudo', 'rep_perm_prénoms',
       'rep_perm_date_naissance', 'rep_perm_ville_naissance',
       'rep_perm_pays_naissance', 'rep_perm_nationalité',
       'rep_perm_adr_ligne1', 'rep_perm_adr_ligne2', 'rep_perm_adr_ligne3',
       'rep_perm_code_postal', 'rep_perm_ville', 'rep_perm_code_commune',
       'rep_perm_pays', 'conjoint_collab_nom_patronym',
       'conjoint_collab_nom_usage', 'conjoint_collab_pseudo',
       'conjoint_collab_prénoms', 'conjoint_collab_date_fin'], axis=1)
    
    # Traitement des cas où le nom et les prénoms sont -
    df_rep.loc[df_rep["prenoms"] == "-", "prenoms"] = ""
    # on retire les termes en trop dans les noms
    re1 = "(INDIVISION SUCCESSORALE DE M\.\s|\sM\.\s|^M\.\s|MME\.\s|MR\.\s|MELLE\.\s|MONSIEUR\.?\s|MADAME\.?\s|MLLE\.\s|REPRESENTEE\sPAR\sMONSIEUR\.?\s|REPRESENTEE\sPAR\s|SUCCESSION\s|SUCCESSION\sDE\s)"
    # df_rep["top_civilite"] = df_rep["nom_patronymique"].str.contains(re1, regex=True)
    df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique"]
    df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique_cor"].str.replace(re1,"")
    
    # Traitement de la qualite
    df_rep["qualite_cor"] = (df_rep["qualite"].str.lower().str.strip())
    # nettoyage de la dénomination
    df_rep["denomination_cor"] = (df_rep["denomination"].str.lower())
    # Correction des adresses
    df_rep.loc[(df_rep["ville"].isna()) & (df_rep["adresse_ligne3"].isna() == False), "ville"] = \
    df_rep.loc[(df_rep["ville"].isna()) & (df_rep["adresse_ligne3"].isna() == False), "adresse_ligne3"]
    df_rep["commune"] = df_rep["ville"]
    df_rep["commune"] = df_rep["commune"].str.lower()
    df_rep["commune"] = (df_rep["commune"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))
    df_rep["commune"] = (df_rep["commune"].str.replace("saint", "st"))
    df_rep["commune"] = df_rep["commune"].str.replace(" ", "")
    # mettre date de naissance sous la forme AAAMMJJ
    df_rep["date_naissance_cor"] = df_rep["date_naissance"].str.replace("-","")
    # traitement des noms et prenoms
    # Traitement des particules
    df_rep["nom_patronymique_cor"] = (df_rep["nom_patronymique_cor"].str.replace(" DE LA ", "-DE-LA-")
        .str.replace(" DU ", "-DU-").str.replace(" LE ", "-LE-").str.replace(" LA ", "-LA-")
        .str.replace(" EL ", "-EL-")
        .str.replace(" SAINT ", "-SAINT-")
        .str.replace(" VAN ", "-VAN-")
        .str.replace(" DE ", "-DE-")
        .str.replace(" D ", "-D-")
        .str.replace("SAINT ", "SAINT-")
    #.str.replace("D ", "D-") -> induit des problemes
        .str.replace("DE ", "DE-")
        .str.replace("DU ", "DU-")
        .str.replace("LE ", "LE-")
        .str.replace("LA ", "LA-")
        .str.replace("EL ", "EL-")
        .str.replace("DI ", "DI-")
        .str.replace("BEN ", "BEN-")
        .str.replace("VAN ", "VAN-")
        .str.replace(" D'", "-D'"))
    df_rep["nom_cor"] = df_rep["nom_patronymique_cor"] 
    df_rep["prenom1_cor"] = df_rep["prenoms"] 
    df_rep["prenom2_cor"] = np.nan 
    df_rep["prenom3_cor"] = np.nan 
    temp = df_rep.loc[df_rep["prenoms"] == "", "nom_patronymique_cor"].str.split(pat=" ", n=3, expand=True)
    temp.columns = ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]
    df_rep.loc[df_rep["prenoms"] == "", ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]] = temp
    df_rep.loc[df_rep["nom_cor"] == "", "nom_cor"] = np.nan
    df_rep.loc[df_rep["prenom1_cor"] == "", "prenom1_cor"] = np.nan
    # correction du type 
    df_rep["type"] = df_rep["type"].str.lower().str.replace(" ", "") 
    # correction des noms prenoms quand prenom est na, type est personne physique et le nom contient une virgule
    temp = df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique")&
            (df_rep["nom_cor"].str.contains(",")),"nom_cor"].str.split(pat=" ", n=3, expand=True)
    temp.columns = ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]
    temp["prenom1_cor"] = temp["prenom1_cor"].str.replace(",", "")
    temp["prenom2_cor"] = temp["prenom2_cor"].str.replace(",", "")
    temp["prenom3_cor"] = temp["prenom3_cor"].str.replace(",", "")
    df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique")&(df_rep["nom_cor"].str.contains(",")),
        ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]] = temp
    # traitement des cas X née Y
    temp = df_rep.loc[(df_rep["nom_cor"].str.contains("NEE\s|née\s|Née\s") == True)&
                        (df_rep["nom_usage"].isna()), "nom_cor"].str.split(pat="NEE\s|née\s|Née\s", n=1, expand=True)
    temp.columns = ["nom_usage_cor", "nom_cor"]
    df_rep.loc[(df_rep["nom_cor"].str.contains("NEE\s|née\s|Née\s") == True)&
                (df_rep["nom_usage"].isna()), ["nom_cor"]] = temp["nom_cor"]
    return df_rep

In [42]:
df_rep = clean_rep_inpi(rep)

/tmp/ipykernel_1382/989057859.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique_cor"].str.replace(re1,"")
/tmp/ipykernel_1382/989057859.py:35: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_rep["top_particule"] = df_rep["nom_patronymique"].str.contains(particule, regex=True)


In [44]:
df_rep.loc[df_rep["top_particule"] == True, "nom_patronymique_cor"].head()

57                     DE-LAFORCADE
75     THIERRY-D'ARGENLIEU-DE-GYVES
108                        DE-MORAS
114                      DE-LORENZO
115                      DE-LORENZO
Name: nom_patronymique_cor, dtype: object

In [45]:
df_rep.columns

Index(['code_greffe', 'nom_greffe', 'numero_gestion', 'siren', 'type',
       'nom_patronymique', 'nom_usage', 'pseudonyme', 'prenoms',
       'denomination', 'siren_representant_pm', 'forme_juridique',
       'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3', 'code_postal',
       'ville', 'code_commune', 'pays', 'date_naissance', 'ville_naissance',
       'pays_naissance', 'nationalite', 'qualite', 'id_representant',
       'date_greffe', 'libelle_evt', 'source', 'year', 'file_path',
       'date_greffe', 'nom_patronymique_cor', 'top_particule', 'qualite_cor',
       'denomination_cor', 'commune', 'date_naissance_cor'],
      dtype='object')

In [67]:
df_rep.loc[df_rep["prenoms"] == "", ].shape # 4705

(4705, 41)

In [78]:
df_rep.shape

(11142886, 41)

In [69]:
df_rep.loc[df_rep["prenoms"] == "", ["nom_cor", "prenom1_cor", "prenom2_cor", "prenom3_cor"]].head()

,nom_cor,prenom1_cor,prenom2_cor,prenom3_cor
24689,SYED-ADIL,RASHID,None,None
25196,ISLAM,AWLAD,None,None
54431,ABD-EL-SALAM,ABD-EL-FATTAH,AHMED-RADWAN,None
70930,PIERRON,None,None,None
70931,PIERRON,None,None,None


In [72]:
df_rep.loc[:,["nom_cor", "prenom1_cor"]].head()

,nom_cor,prenom1_cor
0,BAZILE,Eric
1,BORFIGA,Philippe
2,NaN,NaN
3,GEORGHIOU,Jean-Christophe
4,NaN,NaN


In [79]:
df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["siren_representant_pm"].isna()==False), ].shape

(981443, 41)

In [86]:
df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique"),:].shape # 8526

(8526, 41)

In [85]:
df_rep.loc[(df_rep["prenoms"].isna()==True)&(df_rep["type"]=="p.physique"),:].head()

,code_greffe,nom_greffe,numero_gestion,siren,type,nom_patronymique,nom_usage,pseudonyme,prenoms,denomination,...,nom_patronymique_cor,top_particule,qualite_cor,denomination_cor,commune,date_naissance_cor,nom_cor,prenom1_cor,prenom2_cor,prenom3_cor
674145,8903,Sens,2002D00078,775718463,p.physique,FEDERATION NATIONALE DE REVISION DES COOPERATI...,NaN,NaN,NaN,NaN,...,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,True,commissaire aux comptes suppléant,NaN,paris,NaN,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,NaN,NaN,NaN
683174,8903,Sens,1985D00013,331648063,p.physique,FEDERATION NATIONALE DE REVISION DES COOPERATI...,NaN,NaN,NaN,NaN,...,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,True,commissaire aux comptes suppléant,NaN,coin-les-cuvry,NaN,FEDERATION NATIONALE-DE-REVISION DES COOPERATI...,NaN,NaN,NaN
694286,8501,La Roche-sur-Yon,2007B01050,499451029,p.physique,ERNST & YOUNG ATLANTIQUE,NaN,NaN,NaN,NaN,...,ERNST & YOUNG ATLANTIQUE,False,commissaire aux comptes titulaire,NaN,nantescedex1,NaN,ERNST & YOUNG ATLANTIQUE,NaN,NaN,NaN
694299,8501,La Roche-sur-Yon,1998B00460,419107107,p.physique,FIDEA-FPL,NaN,NaN,NaN,NaN,...,FIDEA-FPL,False,commissaire aux comptes titulaire,NaN,st-gilles-croix-de-vie,NaN,FIDEA-FPL,NaN,NaN,NaN
694828,8501,La Roche-sur-Yon,1983B00115,327017992,p.physique,SELAS G/AUDIT,NaN,NaN,NaN,NaN,...,SELAS G/AUDIT,False,commissaire aux comptes titulaire,NaN,angers,NaN,SELAS G/AUDIT,NaN,NaN,NaN


In [104]:
# traitement du cas avec nee X
df_rep.loc[df_rep["nom_cor"].str.contains("NEE|née|Née") == True,["nom_cor"]].head()

,nom_cor
2084,GIARDINA NEE CUSUMANO PHILIPPINE
2085,GIARDINA NEE CUSUMANO PHILIPPINE
2299,ROMINGER NEE ZAMARON MARIE-CLAIRE ANNIE
3406,SCHNEEGANS
7039,MARCIANO NEE SOUSSAN SARAH


In [106]:
df_rep.loc[df_rep["nom_usage"].isna(), :].shape # 9 300 011

(9300011, 41)

In [108]:
# nom patronymique est censé être le nom de naissance et nom d'usage pas forcément
df_rep.loc[~df_rep["nom_usage"].isna(), ["nom_cor", "nom_usage"]].head()

,nom_cor,nom_usage
35,MOUNIER,BLAZY
51,PEREZ,BENHAMOU
56,LE-CHEVALIER,PALUMBO
78,LEGRIS,PECCI
96,KETCHEDJIAN,BENARD
...,...,...
11142875,HUE,MOQUET
11142876,DELACOUR,CHOL
11142878,LEVASSOR,HUE
11142879,BATAILLE,CORLAY


In [109]:
df_rep.loc[~df_rep["nom_usage"].isna(), ["nom_cor", "nom_usage"]].shape

(1842875, 2)

In [105]:
# donc quand il y a marque nee, il faut mettre ce qu'il y a après nee dans nom_cor et avant dans nom d'usage
df_rep.loc[df_rep["nom_cor"].str.contains("NEE|née|Née") == True,["nom_cor"]].shape # 12 219 cas

(12219, 1)

In [110]:
df_rep.loc[df_rep["nom_cor"].str.contains("NEE|née|Née") == True,["nom_cor"]].head()

,nom_cor
2084,GIARDINA NEE CUSUMANO PHILIPPINE
2085,GIARDINA NEE CUSUMANO PHILIPPINE
2299,ROMINGER NEE ZAMARON MARIE-CLAIRE ANNIE
3406,SCHNEEGANS
7039,MARCIANO NEE SOUSSAN SARAH


In [123]:
df_rep.loc[(df_rep["nom_cor"].str.contains("NEE\s|née\s|Née\s") == True)&(~df_rep["nom_usage"].isna()), ["nom_cor", "nom_usage"]].head()

,nom_cor,nom_usage
50723,GOHIER RAYMONDE-RENEE HENRIETTE,JEUNEMAITRE
564677,NEE BLAIS,AUVRAY
806492,GEIGER NEE TURLAN,GEIGER
1071701,ROBIN NEE BAVOILLOT,ROBIN
1091448,GUEU NEE BOURCEY,GUEU


In [124]:
df_rep.loc[(df_rep["nom_cor"].str.contains("NEE\s|née\s|Née\s") == True)&(~df_rep["nom_usage"].isna()),].shape # seulement 74 sur 8000, on voit que nom d'usage répète
#  parfois ce qui est avant le nee, donc on va garder le nom d'usage quand il est rempli

(74, 41)

In [132]:
df_rep.loc[(df_rep["nom_cor"].str.contains("\sNEE\s|née\s|Née\s") == True)&(df_rep["nom_usage"].isna()),["nom_cor", "nom_usage"]].head()

,nom_cor,nom_usage
273754,MARTIN-LAVIGNE JOSETTE RENEE MARIE-LOUISE,NaN
273755,MARTIN-LAVIGNE JOSETTE RENEE MARIE-LOUISE,NaN
991893,GENSOLLEN CHRISTIANE RENEE MARIE,NaN
2487098,GALMICHE RENNEE GEORGETTE,NaN
2499737,RONCO MURIEL-RENEE JULIETTE,NaN
